In [2]:
from geopy.geocoders import Nominatim
!pip install folium
#for mapping
import folium  # pip install folium (if not installed)
#for making http requests
import requests

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

    100% |████████████████████████████████| 92kB 11.5MB/s ta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [4]:
df = pd.read_csv("Neighbor_uk_coords.csv")

#remove the unwanted column
df = df.drop("Unnamed: 0", axis = 1)

In [5]:
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,GU7 2,South East,Aaron's Hill,51.18291,-0.63098
1,BA8 0,South West,Abbas Combe,51.00283,-2.41825
2,WR6 6,West Midlands,Abberley,52.30522,-2.37574
3,CO5 7,Eastern,Abberton,51.83440,0.91066
4,WR10 2,West Midlands,Abberton,52.17955,-2.00817
5,NE66 2,North East,Abberwick,55.41325,-1.79720
6,CM5 0,Eastern,Abbess End,51.78000,0.28172
7,CM5 0,Eastern,Abbess Roding,51.77815,0.27685
8,EX14 4,South West,Abbey,50.88896,-3.22276
9,GL4 4,South West,Abbeydale,51.84615,-2.19922


In [6]:
loc = "United Kingdom"

g = Nominatim(timeout = 3)

uk = g.geocode(loc)
lat = uk.latitude
long = uk.longitude
print ("Coordinates of United Kingdom are ({}, {}).".format(lat, long))

Coordinates of United Kingdom are (54.7023545, -3.2765753).


In [7]:
uk_map = folium.Map(location = [lat, long], zoom_start = 7)

for latitude, longitude, borough, neighbourhood in zip(df.Latitude, df.Longitude, df.Borough, df.Neighbourhood):
    popup = "{}, {}".format(neighbourhood, borough)
    popup = folium.Popup(popup, parse_html = True)
    folium.CircleMarker([latitude, longitude],
                        radius = 5,
                        tooltip = popup,
                        color = "blue", 
                        fill = True).add_to(uk_map)

uk_map

In [8]:
print("List of unique boroughs:")
for borough in df.Borough.unique():
    print(borough)

List of unique boroughs:
South East
South West
West Midlands
Eastern
North East
Yorkshire and the Humber
North West
London
East Midlands


In [9]:
london_df = df[df.Borough == 'London'].reset_index(drop=True)
london_df.shape

(21, 5)

In [10]:
loc = 'London, England'

geolocator = Nominatim(timeout = 3)
london = geolocator.geocode(loc)
london_lat = london.latitude
london_long = london.longitude
print('The geograpical coordinate of London are {}, {}.'.format(london_lat, london_long))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [11]:
london_map = folium.Map(location = [london_lat, london_long], 
                                  zoom_start = 9)

for lat, lng, label in zip(london_df.Latitude, 
                           london_df.Longitude, 
                           london_df.Neighbourhood):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True).add_to(london_map)  
    
london_map

In [12]:
CLIENT_ID = "VXIUP21EAJ4XFLMR50XI34X35UB5VIQNGLMR544NC0R3AH5U" # your Foursquare ID
CLIENT_SECRET = "0OTIXQYBDBBJMSTHTULSQNUEDDSGBQNKE3VTCPK3HGJZU0WU" # your Foursquare Secret
VERSION = "20180605" # Foursquare API version

print("Your credentails:")
print("CLIENT_ID: " + CLIENT_ID)
print("CLIENT_SECRET:" + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VXIUP21EAJ4XFLMR50XI34X35UB5VIQNGLMR544NC0R3AH5U
CLIENT_SECRET:0OTIXQYBDBBJMSTHTULSQNUEDDSGBQNKE3VTCPK3HGJZU0WU


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["categories"][0]["name"]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Neighbourhood", 
                  "Neighbourhood Latitude", 
                  "Neighbourhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Category"]
    
    return(nearby_venues)

In [14]:
LIMIT = 100
london_venues = getNearbyVenues(names = london_df.Neighbourhood, 
                               latitudes = london_df.Latitude, 
                               longitudes = london_df.Longitude)

In [15]:
london_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbey Wood,51.49083,0.10803,Co-op Food,51.494124,0.112796,Grocery Store
1,Abbey Wood,51.49083,0.10803,Nom Nom Noms,51.493540,0.109896,Fish & Chips Shop
2,Abbey Wood,51.49083,0.10803,Seven Acre Sports,51.491224,0.101388,Soccer Field
3,Abbey Wood,51.49083,0.10803,Bostal Gardens,51.486670,0.110462,Playground
4,Acton,51.50673,-0.26831,Everyone Active,51.506608,-0.266878,Gym / Fitness Center


In [16]:
print(london_venues.shape)

print('The dataframe has {} neighbourhoods.'.format(
        len(london_venues.Neighbourhood.unique())
    )
)

(453, 7)
The dataframe has 21 neighbourhoods.


In [17]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 141 uniques categories.


In [18]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], 
                                         prefix="", 
                                         prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighbourhood'] = london_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,...,Theater,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
london_grouped = london_onehot.groupby('Neighbourhood').mean().reset_index()
london_grouped

,Neighbourhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,...,Theater,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Abbey Wood,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Acton,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000
2,Acton Green,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,0.027027
3,Addington,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Addiscombe,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Alameda Place,0.000000,0.00,0.000000,0.090909,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000
6,Albany Park,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Aldborough Hatch,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Aldersbrook,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Aldgate,0.000000,0.00,0.021277,0.000000,0.00,0.000000,0.021277,0.000000,0.021277,...,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277,0.010638,0.000000


In [21]:
num_top_venues = 8

for hood in london_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbey Wood----
                  venue  freq
0            Playground  0.25
1          Soccer Field  0.25
2     Fish & Chips Shop  0.25
3         Grocery Store  0.25
4     Afghan Restaurant  0.00
5  Outdoor Supply Store  0.00
6             Nightclub  0.00
7                Office  0.00


----Acton----
                  venue  freq
0                   Pub  0.18
1  Gym / Fitness Center  0.09
2     Indian Restaurant  0.05
3                  Café  0.05
4    Chinese Restaurant  0.05
5              Bus Stop  0.05
6         Shopping Mall  0.05
7           Coffee Shop  0.05


----Acton Green----
               venue  freq
0                Pub  0.08
1          Gastropub  0.08
2   Sushi Restaurant  0.05
3           Platform  0.05
4               Park  0.05
5  French Restaurant  0.05
6               Café  0.05
7   Malay Restaurant  0.03


----Addington----
                  venue  freq
0           Bus Station  0.25
1           Gas Station  0.25
2    English Restaurant  0.25
3          Tram Stat

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns = columns)
neighbourhoods_venues_sorted['Neighbourhood'] = london_grouped['Neighbourhood']

for ind in np.arange(london_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], 
                                                                           num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,Playground,Soccer Field,Grocery Store,Fish & Chips Shop,Electronics Store,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service,English Restaurant
1,Acton,Pub,Gym / Fitness Center,Bakery,Coffee Shop,Chinese Restaurant,Café,Bus Stop,Sandwich Place,Fast Food Restaurant,Shopping Mall
2,Acton Green,Pub,Gastropub,French Restaurant,Café,Platform,Park,Sushi Restaurant,Yoga Studio,Middle Eastern Restaurant,Business Service
3,Addington,English Restaurant,Bus Station,Tram Station,Gas Station,Entertainment Service,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,Addiscombe,Park,Fast Food Restaurant,Tram Station,Café,Grocery Store,Pub,Chinese Restaurant,Bakery,English Restaurant,Farmers Market
5,Alameda Place,Bus Stop,Pub,Convenience Store,Fast Food Restaurant,Waterfront,Hotel,Art Gallery,Gym,Burger Joint,Entertainment Service
6,Albany Park,Coffee Shop,Grocery Store,Fast Food Restaurant,Beer Bar,Bakery,Pizza Place,Café,Supermarket,Hotel,Italian Restaurant
7,Aldborough Hatch,Steakhouse,Food & Drink Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service,Yoga Studio
8,Aldersbrook,Soccer Field,Bus Stop,Yoga Studio,Convenience Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service
9,Aldgate,Hotel,Coffee Shop,Cocktail Bar,Salad Place,Restaurant,Gym / Fitness Center,Garden,BBQ Joint,Hotel Bar,Pub


In [37]:
# set number of clusters
kclusters = 8

london_grouped_clustering = london_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, 
                random_state = 0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([8, 3, 3, 9, 3, 3, 3, 2, 4, 3], dtype=int32)

In [38]:
london_merged = london_df

# add clustering labels
london_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), 
                                                       on = 'Neighbourhood')

london_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,SE2 9,London,Abbey Wood,51.49083,0.10803,8,Playground,Soccer Field,Grocery Store,Fish & Chips Shop,Electronics Store,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service,English Restaurant
1,W3 6,London,Acton,51.50673,-0.26831,3,Pub,Gym / Fitness Center,Bakery,Coffee Shop,Chinese Restaurant,Café,Bus Stop,Sandwich Place,Fast Food Restaurant,Shopping Mall
2,W4 5,London,Acton Green,51.49620,-0.27036,3,Pub,Gastropub,French Restaurant,Café,Platform,Park,Sushi Restaurant,Yoga Studio,Middle Eastern Restaurant,Business Service
3,CR0 5,London,Addington,51.35822,-0.03232,9,English Restaurant,Bus Station,Tram Station,Gas Station,Entertainment Service,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,CR0 6,London,Addiscombe,51.37731,-0.07457,3,Park,Fast Food Restaurant,Tram Station,Café,Grocery Store,Pub,Chinese Restaurant,Bakery,English Restaurant,Farmers Market


In [39]:
# create map
map_clusters = folium.Map(location = [london_lat, london_long], zoom_start = 9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], 
                                  london_merged['Longitude'], 
                                  london_merged['Neighbourhood'], 
                                  london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 7,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [40]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Alperton,0,Indian Restaurant,Food & Drink Shop,Clothing Store,Café,Sandwich Place,Middle Eastern Restaurant,Gym / Fitness Center,Dessert Shop,Deli / Bodega,Dim Sum Restaurant


In [41]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Alexandra Park,1,Lake,Deli / Bodega,Golf Course,Pub,Outdoors & Recreation,Restaurant,Café,Yoga Studio,Falafel Restaurant,Event Space


In [42]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Aldborough Hatch,2,Steakhouse,Food & Drink Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service,Yoga Studio


In [43]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Acton,3,Pub,Gym / Fitness Center,Bakery,Coffee Shop,Chinese Restaurant,Café,Bus Stop,Sandwich Place,Fast Food Restaurant,Shopping Mall
2,Acton Green,3,Pub,Gastropub,French Restaurant,Café,Platform,Park,Sushi Restaurant,Yoga Studio,Middle Eastern Restaurant,Business Service
4,Addiscombe,3,Park,Fast Food Restaurant,Tram Station,Café,Grocery Store,Pub,Chinese Restaurant,Bakery,English Restaurant,Farmers Market
5,Alameda Place,3,Bus Stop,Pub,Convenience Store,Fast Food Restaurant,Waterfront,Hotel,Art Gallery,Gym,Burger Joint,Entertainment Service
6,Albany Park,3,Coffee Shop,Grocery Store,Fast Food Restaurant,Beer Bar,Bakery,Pizza Place,Café,Supermarket,Hotel,Italian Restaurant
9,Aldgate,3,Hotel,Coffee Shop,Cocktail Bar,Salad Place,Restaurant,Gym / Fitness Center,Garden,BBQ Joint,Hotel Bar,Pub
10,Aldwych,3,Theater,Coffee Shop,Hotel,Restaurant,Pub,Dessert Shop,Italian Restaurant,Cocktail Bar,Burger Joint,History Museum
14,Angel,3,Coffee Shop,Pub,Gym / Fitness Center,Hotel,Café,Burrito Place,Restaurant,Arts & Crafts Store,Vietnamese Restaurant,Vegetarian / Vegan Restaurant
16,Archway,3,Coffee Shop,Grocery Store,Pub,Bar,Italian Restaurant,Indian Restaurant,Sandwich Place,Café,Seafood Restaurant,Farmers Market


In [44]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Aldersbrook,4,Soccer Field,Bus Stop,Yoga Studio,Convenience Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service


In [45]:
london_merged.loc[london_merged['Cluster Labels'] == 5, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Avery Hill,5,Breakfast Spot,Gym / Fitness Center,Park,Rental Car Location,Café,English Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [46]:
london_merged.loc[london_merged['Cluster Labels'] == 6, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Arkley,6,Bay,Golf Course,Yoga Studio,Entertainment Service,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,English Restaurant


In [47]:
london_merged.loc[london_merged['Cluster Labels'] == 7, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Aperfield,7,Stables,Supermarket,Chinese Restaurant,Coffee Shop,Grocery Store,Fish & Chips Shop,Creperie,Cupcake Shop,Dance Studio,Gym / Fitness Center


In [48]:
london_merged.loc[london_merged['Cluster Labels'] == 8, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,8,Playground,Soccer Field,Grocery Store,Fish & Chips Shop,Electronics Store,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service,English Restaurant


In [49]:
london_merged.loc[london_merged['Cluster Labels'] == 9, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Addington,9,English Restaurant,Bus Station,Tram Station,Gas Station,Entertainment Service,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [50]:
london_merged.loc[london_merged['Cluster Labels'] == 10, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Anerley,10,Hardware Store,Grocery Store,Pub,Park,Gas Station,Electronics Store,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service


In [51]:
london_merged.loc[london_merged['Cluster Labels'] == 11, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Arnos Grove,11,Bus Stop,Metro Station,Entertainment Service,Park,Pool,Turkish Restaurant,Chinese Restaurant,Beer Bar,Fast Food Restaurant,Farmers Market


In [53]:
london_merged.loc[london_merged['Cluster Labels'] == 12, london_merged.columns[[2] + list(range(5, london_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Ardleigh Green,12,Home Service,Gym / Fitness Center,Grocery Store,Pizza Place,Pub,Electronics Store,Farmers Market,Falafel Restaurant,Event Space,Entertainment Service
